<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=38d630394340c81278f3e0322a8ddb21fb563c9c68491ec95ea8f3808311a1a8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-19 14:16:23
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: 70.99 K (0.51%)
Current PnL: -27.10 L (-17.5%)
CY Booked + Current PnL: -12.81 L (-8.27%)
-------------------
Total profit:  1.36 L
Total loss:  -28.46 L
-------------------
Total Booked + Current PnL: 14.17 L (11.09%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.21%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.73 L (65.54%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.02%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.31,-20.39,25.73,0.09,20642.0,-20553.0,80224.0,86.13,26.0,M-SC,2.31,253.0,-1.00,0.58,5.40,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-2.35,7.07,11.78,19.69,24185.0,13563.0,205305.0,-3.43,50.0,X-MC,2.50,78.0,0.56,1.47,14.35,XY25,NTT,AC
50,MASFIN,398.61,1.49,-2.89,25.69,22.05,24443.0,-2835.0,95145.0,-16.44,52.0,H-SC,6.41,164.0,-0.12,0.68,38.30,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.32,-16.00,19.07,0.02,27962.0,-27929.0,146630.0,123.77,42.0,M-SC,13.11,234.0,-1.00,1.05,27.54,OX40N,NTT,PAINTS
43,ITC,452.00,0.40,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,47.0,X-LC,2.13,5.0,-0.12,1.70,3.69,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.0,8.57,-21.56,68.31,32.03,70644.0,-28418.0,103417.0,-15.03,63.0,H-SC,9.50,148.0,-0.40,0.74,15.84,OX40N,NTT,IT
6,ASIANTILES,137.0,4.97,-4.40,90.04,81.67,80748.0,-4130.0,89680.0,8072.22,71.0,L-SC,13.01,271.0,-0.05,0.64,73.67,XR,NTT,CERAMICS
57,RAJOOENG,143.1,4.24,-34.56,113.33,39.61,76022.0,-35420.0,67080.0,-55.65,40.0,H-SC,30.32,135.0,-0.47,0.48,5.69,AR,ATH,MISC
10,BANDHANBNK,400.0,3.09,-27.93,166.99,92.42,335247.0,-77800.0,200759.0,41.52,51.0,H-SC,9.24,170.0,-0.23,1.44,16.02,XY24,NTT,BANKS
64,SHALBY,327.0,2.91,-24.43,65.54,25.10,89721.0,-44248.0,136895.0,906.38,40.0,M-SC,11.00,250.0,-0.49,0.98,9.71,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-3.26,-38.31,256.13,119.69,427814.0,-103731.0,167030.0,-37.58,40.0,H-SC,7.51,156.0,-0.24,1.20,31.95,AR,ATH,IT
82,VOLTAS,1530.00,-2.35,7.07,11.78,19.69,24185.0,13563.0,205305.0,-3.43,50.0,X-MC,2.50,78.0,0.56,1.47,14.35,XY25,NTT,AC
38,INDIAMART,4810.62,-2.08,-4.73,117.00,106.72,137470.0,-5840.0,117496.0,-52.92,36.0,H-SC,3.68,139.0,-0.04,0.84,17.54,AR,ATH,MISC
61,ROUTE,2227.21,-1.32,-45.33,209.21,69.05,152200.0,-60316.0,72750.0,-55.87,56.0,H-SC,21.95,140.0,-0.40,0.52,12.20,SR,ATH,IT
32,HCLTECH,1922.01,-1.22,6.51,17.11,24.73,44087.0,15744.0,257668.0,11.06,52.0,X-LC,6.90,7.0,0.36,1.85,22.73,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.9,-0.98,109.97,107.91,160773.0,-1447.0,146197.0,-19.34,59.0,M-SC,12.29,216.0,-0.01,1.05,9.01,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.90,-0.98,109.97,107.91,160773.0,-1447.0,146197.0,-19.34,59.0,M-SC,12.29,216.0,-0.01,1.05,9.01,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.32,-16.00,19.07,0.02,27962.0,-27929.0,146630.0,123.77,42.0,M-SC,13.11,234.0,-1.00,1.05,27.54,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.31,-20.39,25.73,0.09,20642.0,-20553.0,80224.0,86.13,26.0,M-SC,2.31,253.0,-1.00,0.58,5.40,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,8.57,-21.56,68.31,32.03,70644.0,-28418.0,103417.0,-15.03,63.0,H-SC,9.50,148.0,-0.40,0.74,15.84,OX40N,NTT,IT
66,SIS,528.00,0.80,-23.64,58.80,21.26,49855.0,-26244.0,84788.0,2001.04,53.0,H-SC,4.83,166.0,-0.53,0.61,14.56,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.2,8.44,58.86,72.27,108152.0,14296.0,183744.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.13,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.20,8.44,58.86,72.27,108152.0,14296.0,183744.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.13,XR,NTT,IT
25,FINCABLES,1641.55,0.90,-0.98,109.97,107.91,160773.0,-1447.0,146197.0,-19.34,59.0,M-SC,12.29,216.0,-0.01,1.05,9.01,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-2.08,-4.73,117.00,106.72,137470.0,-5840.0,117496.0,-52.92,36.0,H-SC,3.68,139.0,-0.04,0.84,17.54,AR,ATH,MISC
6,ASIANTILES,137.00,4.97,-4.40,90.04,81.67,80748.0,-4130.0,89680.0,8072.22,71.0,L-SC,13.01,271.0,-0.05,0.64,73.67,XR,NTT,CERAMICS
37,IEX,219.00,0.87,-4.32,55.34,48.63,105871.0,-8644.0,191310.0,-36.24,48.0,H-SC,13.54,136.0,-0.08,1.37,6.73,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,0.07,-11.33,44.47,28.11,79423.0,-22810.0,178600.0,-35.73,24.0,X-MC,13.79,55.0,-0.29,1.28,0.07,X40,ATH,APPARELS
9,BAJAJHFL,181.50,0.17,-18.45,90.65,55.47,179847.0,-44889.0,198397.0,-27.82,27.0,X-MC,14.37,64.0,-0.25,1.42,1.66,X40N,ATH,FINANCE
8,AWL,485.00,0.44,-23.95,101.83,53.50,233686.0,-72265.0,229486.0,-63.25,30.0,X-MC,4.21,58.0,-0.31,1.65,1.93,XY24,NTT,FMCG
11,BATAINDIA,2096.00,0.41,-39.11,122.03,35.20,95613.0,-50318.0,78352.0,2.67,32.0,X-SC,16.62,92.0,-0.53,0.56,0.50,X40,NTT,FOOTWEAR
33,HINDUNILVR,2922.00,0.71,-11.24,28.07,13.67,64045.0,-28903.0,228160.0,-17.95,34.0,X-LC,2.60,14.0,-0.45,1.64,7.71,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.35,-1.09,25.84,24.48,64372.0,-2739.0,249117.0,-81.09,46.0,X-SC,0.76,86.0,-0.04,1.79,21.80,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.28,-4.18,26.43,21.14,57572.0,-9510.0,217829.0,-11.03,39.0,X-MC,1.34,74.0,-0.17,1.56,22.78,XY24,NTT,PAINTS
20,DABUR,735.0,0.16,-3.54,48.98,43.70,117197.0,-8793.0,239275.0,-11.44,35.0,X-MC,2.02,72.0,-0.08,1.72,10.97,XY24,BTT,FMCG
43,ITC,452.0,0.40,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,47.0,X-LC,2.13,5.0,-0.12,1.70,3.69,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.02,3.24,16.98,20.77,41046.0,7585.0,241729.0,-5.80,44.0,X-MC,2.27,62.0,0.18,1.74,10.38,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,0.07,-11.33,44.47,28.11,79423.0,-22810.0,178600.0,-35.73,24.0,X-MC,13.79,55.0,-0.29,1.28,0.07,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.41,-39.11,122.03,35.20,95613.0,-50318.0,78352.0,2.67,32.0,X-SC,16.62,92.0,-0.53,0.56,0.50,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.63,-25.78,121.18,64.15,214004.0,-61351.0,176600.0,-56.75,35.0,X-SC,3.18,82.0,-0.29,1.27,0.63,XY24,BTT,CEMENT
19,COLPAL,3726.84,0.51,-20.24,77.41,41.51,162615.0,-53295.0,210070.0,-4.98,38.0,X-MC,7.63,61.0,-0.33,1.51,0.64,XY25,ATH,FMCG
59,RELAXO,1176.00,-0.17,-47.94,192.54,52.31,145514.0,-69584.0,75576.0,-44.54,41.0,X-SC,7.30,91.0,-0.48,0.54,1.16,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.36,-29.76,103.36,42.84,47196.0,-19344.0,45662.0,-54.27,44.0,X-SC,34.78,83.0,-0.41,0.33,3.30,XY24,NTT,MISC
59,RELAXO,1176.00,-0.17,-47.94,192.54,52.31,145514.0,-69584.0,75576.0,-44.54,41.0,X-SC,7.30,91.0,-0.48,0.54,1.16,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.41,-39.11,122.03,35.20,95613.0,-50318.0,78352.0,2.67,32.0,X-SC,16.62,92.0,-0.53,0.56,0.50,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,1.68,-2.14,26.79,24.08,33282.0,-2718.0,124232.0,-10.14,43.0,X-SC,5.53,89.0,-0.08,0.89,17.05,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,1.09,-16.52,71.69,43.32,97470.0,-26912.0,135960.0,-28.31,40.0,X-SC,5.05,90.0,-0.28,0.98,4.39,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.39,-9.44,33.29,20.71,114026.0,-35713.0,342524.0,-22.01,70.0,X-LC,3.87,1.0,-0.31,2.46,14.45,X40,ATH,IT
41,INFY,1972.00,0.44,11.23,20.69,34.24,72006.0,35133.0,348021.0,-13.50,68.0,X-LC,4.87,2.0,0.49,2.50,20.44,X40,NTT,IT
75,TMPV,600.00,1.46,-28.09,71.01,22.98,115351.0,-63454.0,162444.0,-25.52,39.0,X-LC,6.84,3.0,-0.55,1.17,2.17,XY24,NTT,AUTO
81,VBL,671.64,-0.73,-5.03,42.86,35.67,128561.0,-15886.0,299956.0,-16.39,49.0,X-LC,3.94,4.0,-0.12,2.15,8.13,X40N,ATH,FMCG
43,ITC,452.00,0.40,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,47.0,X-LC,2.13,5.0,-0.12,1.70,3.69,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.35,-37.53,112.72,32.89,54368.0,-28974.0,48233.0,-695.74,54.0,L-MC,6.36,259.0,-0.53,0.35,32.83,XR,NTT,BANKS
6,ASIANTILES,137.00,4.97,-4.40,90.04,81.67,80748.0,-4130.0,89680.0,8072.22,71.0,L-SC,13.01,271.0,-0.05,0.64,73.67,XR,NTT,CERAMICS
50,MASFIN,398.61,1.49,-2.89,25.69,22.05,24443.0,-2835.0,95145.0,-16.44,52.0,H-SC,6.41,164.0,-0.12,0.68,38.30,XR,ATH,FINANCE
13,BSOFT,831.70,-0.08,-21.50,92.08,50.79,101670.0,-30238.0,110415.0,0.89,65.0,H-SC,6.88,151.0,-0.30,0.79,28.56,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.32,-16.00,19.07,0.02,27962.0,-27929.0,146630.0,123.77,42.0,M-SC,13.11,234.0,-1.00,1.05,27.54,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.08,-21.50,92.08,50.79,101670.0,-30238.0,110415.0,0.89,65.0,H-SC,6.88,151.0,-0.30,0.79,28.56,XR,ATH,IT
84,WIPRO,420.00,0.20,8.44,58.86,72.27,108152.0,14296.0,183744.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.13,XR,NTT,IT
6,ASIANTILES,137.00,4.97,-4.40,90.04,81.67,80748.0,-4130.0,89680.0,8072.22,71.0,L-SC,13.01,271.0,-0.05,0.64,73.67,XR,NTT,CERAMICS
41,INFY,1972.00,0.44,11.23,20.69,34.24,72006.0,35133.0,348021.0,-13.50,68.0,X-LC,4.87,2.0,0.49,2.50,20.44,X40,NTT,IT
61,ROUTE,2227.21,-1.32,-45.33,209.21,69.05,152200.0,-60316.0,72750.0,-55.87,56.0,H-SC,21.95,140.0,-0.40,0.52,12.20,SR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.37
1,25,44.86
2,50,76.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.92
MC    30.83
LC    24.24
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.70
X40      23.10
X40N     12.99
XR        9.82
XY25      9.07
AR        8.96
OX40N     7.58
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.35
H-SC    23.73
X-LC    20.79
M-SC    11.63
X-SC     8.08
H-MC     4.79
L-SC     1.48
M-MC     1.34
M-LC     1.32
H-LC     1.11
L-LC     1.02
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.30,-8.33,43.16
IT,13.16,-14.65,72.30
FINANCE,9.45,-20.85,73.41
MISC,7.27,-28.16,80.58
ELECTRICAL,5.99,-12.47,53.66
PAINTS,5.82,-13.79,30.70
INSURANCE,4.80,-1.32,35.99
PHARMA,4.11,-2.58,34.87
AUTO,2.86,-31.98,77.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3200870.0,21
AR,1322632.0,10
XR,1295105.0,13
X40,1005125.0,14
X40N,876268.0,9
OX40N,736861.0,10
XY25,378377.0,6
SR,279039.0,2
MH,78953.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3670787.0,25
M-SC,1440556.0,15
X-MC,1434544.0,16
X-LC,830085.0,11
X-SC,794621.0,8
H-MC,410446.0,3
L-SC,254996.0,3
M-LC,108152.0,1
H-LC,78551.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1286593.0      6
           AR         903918.0      5
M-SC       XY24       824042.0      6
H-SC       XR         787632.0      7
X-MC       X40        493894.0      7
           XY24       408455.0      3
           X40N       345395.0      4
H-SC       OX40N      334652.0      4
X-LC       X40        318448.0      5
M-SC       OX40N      316086.0      5
X-SC       X40N       307356.0      3
           XY24       294482.0      3
H-SC       SR         279039.0      2
X-LC       X40N       223517.0      2
H-MC       AR         215815.0      2
           XY24       194631.0      1
X-SC       X40        192783.0      2
X-LC       XY24       192667.0      2
X-MC       XY25       186800.0      2
M-SC       XR         176080.0      2
L-SC       XR         168873.0      2
M-SC       AR         124348.0      2
M-LC       XR         108152.0      1
X-LC       XY25        95453.0      2
L-SC       OX40N       86123.0      1
H-SC       MH          78953.0      1
H-LC       AR          78551.0      1
M-MC       XY25        59243.0      1
L-MC       XR          54368.0      1
L-LC       XY25        36881.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
